In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./ml-100k/u.data',sep='\t',names=['userID', 'movieID', 'rating', 'time'])

In [3]:
df.drop('time', axis='columns', inplace=True)

In [4]:
df

,userID,movieID,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [5]:
average_user_ratings = df['rating'].count() / len(df['userID'].unique())

In [6]:
average_film_ratings = df['rating'].count() /len(df['movieID'].unique())

In [7]:
completeness = df['rating'].count() / ( len(df['userID'].unique()) *  len(df['movieID'].unique()))

In [8]:
muserID = 303

In [9]:
means = df.groupby(['userID'])['rating'].mean()

In [10]:
means

userID
1      3.610294
2      3.709677
3      2.796296
4      4.333333
5      2.874286
         ...   
939    4.265306
940    3.457944
941    4.045455
942    4.265823
943    3.410714
Name: rating, Length: 943, dtype: float64

In [11]:
user_movies = df[['movieID','rating']].where(df['userID']==muserID).dropna()
#input_movies = df[['movieID','rating']]
user_movies

,movieID,rating
15,785.0,3.0
161,919.0,4.0
233,69.0,5.0
246,842.0,2.0
260,815.0,3.0
...,...,...
99123,1217.0,1.0
99699,627.0,3.0
99799,939.0,3.0
99823,1086.0,1.0


In [12]:
user_subset = df[df['movieID'].isin(user_movies['movieID'].tolist())]
#print(user_subset.shape)


In [13]:
from math import sqrt

In [14]:
user_subset_group = user_subset.groupby(['userID'])

# let's look at one of the users, e.g. the one with userID=1130
tt = user_subset_group.get_group(1)

In [15]:
tt

,userID,movieID,rating
333,1,33,4
334,1,160,4
639,1,202,5
687,1,171,5
933,1,155,2
...,...,...,...
92049,1,28,4
92487,1,172,5
94019,1,122,3
96699,1,152,5


In [16]:
temp_df = user_movies[user_movies['movieID'].isin(tt['movieID'].tolist())]

In [17]:
temp_df = temp_df.merge(tt,on='movieID',how='inner')

In [18]:
temp_df

,movieID,rating_x,userID,rating_y
0,69.0,5.0,1,3
1,134.0,5.0,1,4
2,252.0,3.0,1,2
3,200.0,4.0,1,3
4,161.0,5.0,1,4
...,...,...,...,...
167,259.0,3.0,1,1
168,81.0,4.0,1,5
169,116.0,5.0,1,3
170,191.0,5.0,1,5


In [19]:
def calculate_persona_corr(user_subset_group, input_movies):
    pearson_corr_dict = {}
    r_mean_muser = means[muserID]

    for name, tt in user_subset_group:

        r_mean_user= means[name]

        temp_df = user_movies[user_movies['movieID'].isin(tt['movieID'].tolist())]
        temp_df = temp_df.merge(tt,on='movieID',how='inner')
        

        korr = min(temp_df.count()[0]/50,1)

        temp_df['Smu'] = (temp_df['rating_x']-r_mean_muser)
        temp_df['Su'] = (temp_df['rating_y']-r_mean_user)

        
        muser_sum = sqrt(  (temp_df['Smu']*temp_df['Smu']).sum() )
        user_sum = sqrt(  (temp_df['Su']*temp_df['Su']).sum() )



        if(user_sum!=0 and muser_sum!=0):
            s = korr * ( (temp_df['Su']*temp_df['Smu']).sum()) /  (user_sum * muser_sum )
        else:
            s = 0


        pearson_corr_dict[name]=s
        

    return pearson_corr_dict


pearson_corr_dict = calculate_persona_corr(user_subset_group, user_movies)
pearson_corr_dict.items()

15.348933677730559
15.012386754122852
5.7558108446692415
4.765467177450165
6.523242635996412
5.878047322812067
4.776176260868687
3.741657386773941
12.803392766539138
13.958883061568022
13.208370172736664
11.783462178319501
17.440285436584897
15.32664476295835
8.064295208884479
7.687837407902293
4.638192763921381
1.9412869549147842
12.725062193993177
5.738677518360423
12.255653942089538
9.694859118965537
7.7874532078190395
5.053767042768023
20.71615366345175
22.607008129817235
10.331667504585718
9.133188902305582
8.265904316433328
10.178101492911141
12.257538009908018
9.328352655496369
5.948731396851186
4.967239614489632
14.18943914991402
10.353418015641138
5.43246847029594
3.152776554086889
8.299711761259653
6.901558549825297
11.282270687044676
12.651612304551726
11.010923373758931
13.370223573970959
12.925975026032205
10.152287640948039
9.339668242260355
5.995023888082943
8.263063156272054
4.418596764828311
10.724994664453586
7.335049571945753
4.888777294255244
3.6771728270506947
9.39

11.977751912539985
12.283899597120275
4.502988614387926
3.6996621467371855
3.542278697631783
2.4710038675448383
3.329235650987684
2.7608875384557052
8.840062855557782
5.026313931000284
13.599766505325938
12.505837913637802
5.0347839832042665
3.5326107686773547
13.686665768374748
15.516369163936458
5.6133302781586965
4.220266684413908
9.187023606091582
8.12780563804253
13.196364127803005
9.273212881309561
11.560997815074545
10.689857102899387
9.236025741880468
6.8206427222472055
4.997451710433516
2.4660966430902955
10.547088204763961
7.010771664556901
11.004267812139007
10.431272186119438
9.45280945998076
10.224508395334187
14.801459661607696
11.051165747116018
5.9498059119605475
4.693957924527011
4.619119461478844
5.020750890903334
6.953435273763142
4.528631870251443
3.706863204003598
3.1909703330909656
5.2538198267230865
3.428277471231447
11.701012465263267
8.586758519662702
11.161622101496553
8.996793665091515
11.007730858434511
8.842448127576937
6.656382868640296
6.261775296804801
5

7.525102561940322
5.384907605233121
4.752247571095433
6.999777014435605
16.479514601755103
11.32337777265337
5.893248716836465
4.916942803268332
15.319732023212039
11.143900407241079
4.549094616866977
4.044749683231337
9.62606099627158
7.5254057529320075
5.343681442334276
2.263282788250694
11.756322531261777
11.745186619388134
13.672450684675447
13.266427115945863
8.733392134424173
6.05273124759026
8.53114310256439
5.723820170737266
4.150808962935589
2.786006877149916
6.487937560479111
5.9576210912041265
11.795833815807576
10.489719323210563
3.9179290692996704
3.640054944640259
11.296431628153217
10.281831032589285
14.371465230615966
14.109289054749683
12.57445247573606
9.334487295037928
7.900142123879031
5.911418608121155
7.011730075406884
8.406566487621644
6.005045419227785
5.946501622189278
8.00704874182693
4.645659130880199
12.850336459451102
11.534981684820128
8.419468640541286
6.8785954012742945
13.771050660050406
14.15835802732861
9.854613429586935
10.768052957025581
17.05324152

11.337573794111192
8.308935478095226
4.473315134109299
2.2228610192972282
7.99385665655894
6.450663044913156
13.184916829251383
11.68986201342568
14.68790957529687
13.085349779447462
9.370027424735994
8.956188891425693
5.53817756706049
5.4349297638940595
7.7958917962154475
7.92939156635669
3.850859458761881
2.223610677354389
12.592349593609812
11.243025048664526
6.7940997194538335
5.049864729559404
5.432468470295939
3.2903565583370646
6.66362770249831
6.432766467785516
4.791490700856862
4.28485705712571
4.547320240701222
5.0
17.501373412985487
14.860745268407207
6.209261740106287
3.311268192312444
2.966888912711
2.325545799442815
7.749370390248261
6.387910586572878
9.273628622534991
9.263117285737293
3.813856513114986
4.12189276910499
7.217991988052793
8.269200917572409
7.753782365657241
7.15689816933635
7.681474469835568
6.477334033116966
6.122318846721788
4.898979485566356
6.01732601601691
5.468080849131206
12.350765297012869
8.657807650264855
7.435226462477335
6.729041536504289
5.06

dict_items([(1, 0.4245613574674966), (2, 0.16016031955929916), (3, 0.11798295948018063), (4, 0.024556404988481384), (5, 0.32918822299274386), (6, 0.1816781561218402), (7, 0.30818796941981147), (8, 0.454070238946531), (9, 0.10805706885963821), (10, 0.19333518800541102), (11, 0.3100905581187021), (12, 0.0453942001305033), (13, 0.30012105953555285), (14, -0.08068821208423799), (15, 0.07278452538907516), (16, 0.45294511177024466), (17, 0.2869319963019883), (18, 0.10199533716173226), (19, 0.017317032792031763), (20, -0.14840767880687167), (21, 0.43839608110355494), (22, 0.4833783457679913), (23, 0.08995907822737882), (24, 0.2257670370538249), (25, 0.17851246741545296), (26, 0.43932007585734223), (27, 0.1284456126447041), (28, 0.30737804550267145), (29, 0.2944154141441384), (30, 0.293536281060071), (31, 0.021593366435969232), (32, 0.285749428405663), (33, -0.07186386697675598), (34, 0.031601608982286075), (35, -0.045995460911177975), (36, -0.09405071042273977), (37, 0.3436679869495456), (38,

In [20]:
pearson_corr_dict

{1: 0.4245613574674966,
 2: 0.16016031955929916,
 3: 0.11798295948018063,
 4: 0.024556404988481384,
 5: 0.32918822299274386,
 6: 0.1816781561218402,
 7: 0.30818796941981147,
 8: 0.454070238946531,
 9: 0.10805706885963821,
 10: 0.19333518800541102,
 11: 0.3100905581187021,
 12: 0.0453942001305033,
 13: 0.30012105953555285,
 14: -0.08068821208423799,
 15: 0.07278452538907516,
 16: 0.45294511177024466,
 17: 0.2869319963019883,
 18: 0.10199533716173226,
 19: 0.017317032792031763,
 20: -0.14840767880687167,
 21: 0.43839608110355494,
 22: 0.4833783457679913,
 23: 0.08995907822737882,
 24: 0.2257670370538249,
 25: 0.17851246741545296,
 26: 0.43932007585734223,
 27: 0.1284456126447041,
 28: 0.30737804550267145,
 29: 0.2944154141441384,
 30: 0.293536281060071,
 31: 0.021593366435969232,
 32: 0.285749428405663,
 33: -0.07186386697675598,
 34: 0.031601608982286075,
 35: -0.045995460911177975,
 36: -0.09405071042273977,
 37: 0.3436679869495456,
 38: 0.17791515621598578,
 39: 0.06665623478975442,
 

In [21]:
def create_pearson_df(pearson_corr_dict):
    pearson_df = pd.DataFrame.from_dict(pearson_corr_dict, orient='index')
    pearson_df.columns = ['similarityIndex']
    pearson_df['userId'] = pearson_df.index
    pearson_df.index = range(len(pearson_df))
    return pearson_df


pearson_df = create_pearson_df(pearson_corr_dict)


In [22]:
pearson_df = pearson_df[pearson_df['userId']!=muserID]

In [23]:
pearson_df

,similarityIndex,userId
0,0.424561,1
1,0.160160,2
2,0.117983,3
3,0.024556,4
4,0.329188,5
...,...,...
938,0.169851,939
939,0.130840,940
940,0.189717,941
941,0.144103,942


In [24]:
top_users = pearson_df.sort_values(by='similarityIndex', ascending=False)[0:30]
top_users

,similarityIndex,userId
591,0.661205,592
396,0.656966,397
275,0.650072,276
639,0.646231,640
599,0.637041,600
366,0.597316,367
267,0.581008,268
266,0.573335,267
594,0.573129,595
456,0.570155,457


In [25]:
top_users_rating = top_users.merge(df, left_on='userId', right_on='userID', how='inner')
top_users_rating = top_users_rating.merge(means, left_on='userId', right_on='userID', how='inner')
top_users_rating.head()

,similarityIndex,userId,userID,movieID,rating_x,rating_y
0,0.661205,592,592,418,4,3.813889
1,0.661205,592,592,124,5,3.813889
2,0.661205,592,592,50,5,3.813889
3,0.661205,592,592,204,5,3.813889
4,0.661205,592,592,297,5,3.813889


In [26]:
tt = top_users_rating.groupby('movieID')

In [27]:
top_users_rating.groupby('movieID').count()

,similarityIndex,userId,userID,rating_x,rating_y
movieID,,,,,
1,17,17,17,17,17
2,8,8,8,8,8
3,11,11,11,11,11
4,13,13,13,13,13
5,6,6,6,6,6
...,...,...,...,...,...
1609,1,1,1,1,1
1620,1,1,1,1,1
1623,1,1,1,1,1


In [28]:
def outRating(users):
    
    r_mean = means[muserID]
    films = {}
    
    for name,i in users:
        i['Sx'] = i['similarityIndex'] * (i['rating_x']-i['rating_y'])
        Sy = i['similarityIndex'].abs().sum()
        films[name] = r_mean+ i['Sx'].sum() / Sy

        
    return films
        

In [35]:
d = outRating(tt)

In [36]:
d = pd.DataFrame.from_dict(d, orient='index')
d.columns = ['raitng']
d['moviesID'] = d.index



In [37]:
d

,raitng,moviesID
1,4.130402,1
2,3.148397,2
3,2.921983,3
4,3.885640,4
5,3.375182,5
...,...,...
1609,0.551814,1609
1620,0.551814,1620
1623,3.551814,1623
1675,2.906372,1675


In [38]:
d=d[~d['moviesID'].isin(user_movies['movieID'].tolist())]

In [39]:
d

,raitng,moviesID
6,2.350395,6
10,3.626110,10
14,4.048624,14
16,3.129603,16
20,3.837050,20
...,...,...
1609,0.551814,1609
1620,0.551814,1620
1623,3.551814,1623
1675,2.906372,1675


In [40]:
d = d[~d['moviesID'].isin(user_movies['movieID'].tolist())]

top_films = d.sort_values(by='raitng', ascending=False)[0:10]

In [ ]:
top_films

In [41]:
out = {
    "average_film_ratings": average_film_ratings,
    "average_user_ratings": average_user_ratings,
    "completeness": completeness,
    "pearson_neighbours": top_users['userId'][0:30].tolist(),
    "pearson_top10": top_films['moviesID'].tolist()
}

In [42]:
out

{'average_film_ratings': 59.45303210463734,
 'average_user_ratings': 106.04453870625663,
 'completeness': 0.06304669364224531,
 'pearson_neighbours': [592,
  397,
  276,
  640,
  600,
  367,
  268,
  267,
  595,
  457,
  195,
  294,
  323,
  291,
  381,
  532,
  758,
  624,
  715,
  831,
  560,
  453,
  82,
  893,
  776,
  615,
  526,
  63,
  851,
  293],
 'pearson_top10': [1137, 205, 524, 481, 45, 513, 1050, 114, 611, 641]}

In [43]:
import json
with open("lab06.json", "w") as outfile:
    json.dump(out, outfile)
